In [ ]:
!ls ${TF_OBJECT_DETECTION_API}/samples/configs/*pets.config

# Not Pretrained

In [ ]:
%%bash
#export MODEL_NAME='ssd_mobilenet_v1'
#export MODEL_NAME='faster_rcnn_resnet152'
#export MODEL_NAME='faster_rcnn_resnet50'
export MODEL_NAME='faster_rcnn_inception_resnet_v2_atrous'

export SRC_PIPELINE=${TF_OBJECT_DETECTION_API}/samples/configs/${MODEL_NAME}_pets.config

export NUM_CLASS=37

export TFRECORD_PATH='/data/pets/pet_faces_tfrecord' # http://download.tensorflow.org/models/object_detection/pet_faces_tfrecord.tar.gz
export MODEL_OUT_PATH="/data/pets/trained_model"
mkdir -p ${MODEL_OUT_PATH}

cp ${SRC_PIPELINE} ${MODEL_OUT_PATH}/pipeline.config

sed -i "s|fine_tune_checkpoint|#fine_tune_checkpoint|g" ${MODEL_OUT_PATH}/pipeline.config
sed -i "s|PATH_TO_BE_CONFIGURED/[a-z_]*_label_map.pbtxt|${TF_OBJECT_DETECTION_API}/data/pet_label_map.pbtxt|g" ${MODEL_OUT_PATH}/pipeline.config
sed -i "s|PATH_TO_BE_CONFIGURED/[a-z_]*_train|${TFRECORD_PATH}/pet_faces_train|g" ${MODEL_OUT_PATH}/pipeline.config
sed -i "s|PATH_TO_BE_CONFIGURED/[a-z_]*_val|${TFRECORD_PATH}/pet_faces_val|g" ${MODEL_OUT_PATH}/pipeline.config
sed -i "s|num_classes: [[:digit:]]\+|num_classes: ${NUM_CLASS}|g" ${MODEL_OUT_PATH}/pipeline.config

sed -i "s|.record\"|.record-?????-of-00010\"|g" ${MODEL_OUT_PATH}/pipeline.config 
sed -i "s|step: 0|step: 1|g" ${MODEL_OUT_PATH}/pipeline.config # Bug fix 

# Check the final config manually
#cat ${MODEL_OUT_PATH}/pipeline.config


In [ ]:
cat /data/pets/trained_model/pipeline.config

# With Pretrained

In [ ]:
%%bash
# find preatrained models from https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md 
#export MODEL_NAME='ssd_mobilenet_v1_coco_2018_01_28' 
export MODEL_NAME='ssd_inception_v2_coco_2018_01_28'

export PRETRAINED_MODEL_PATH="/data/pretrained_models/${MODEL_NAME}"
export SRC_PIPELINE=${PRETRAINED_MODEL_PATH}/pipeline.config
#export SRC_PIPELINE=${TF_OBJECT_DETECTION_API}/samples/configs/ssd_inception_v2_pets.config

mkdir -p /data/pretrained_models
cd /data/pretrained_models

if [[ -d /data/pretrained_models/${MODEL_NAME} ]]
then 
  echo "Model ${MODEL_NAME} already exists" 
else
  echo "Downloading model ${MODEL_NAME}" 
  wget -c --show-progress --progress=bar http://download.tensorflow.org/models/object_detection/${MODEL_NAME}.tar.gz 
  tar xvzf ${MODEL_NAME}.tar.gz
fi

echo "Model path: /data/pretrained_models/${MODEL_NAME}"

 
export NUM_CLASS=37

export TFRECORD_PATH='/data/pets/tfrecord' # http://download.tensorflow.org/models/object_detection/pet_faces_tfrecord.tar.gz
export MODEL_OUT_PATH="/data/pets/trained_model"
mkdir -p ${MODEL_OUT_PATH}

cp ${SRC_PIPELINE} ${MODEL_OUT_PATH}/pipeline.config

sed -i "s|PATH_TO_BE_CONFIGURED/model.ckpt|${PRETRAINED_MODEL_PATH}/model.ckpt|g" ${MODEL_OUT_PATH}/pipeline.config
sed -i "s|PATH_TO_BE_CONFIGURED/[a-z_]*_label_map.pbtxt|${TF_OBJECT_DETECTION_API}/data/pet_label_map.pbtxt|g" ${MODEL_OUT_PATH}/pipeline.config
sed -i "s|PATH_TO_BE_CONFIGURED/[a-z_]*_train|${TFRECORD_PATH}/pet_faces_train|g" ${MODEL_OUT_PATH}/pipeline.config
sed -i "s|PATH_TO_BE_CONFIGURED/[a-z_]*_val|${TFRECORD_PATH}/pet_faces_val|g" ${MODEL_OUT_PATH}/pipeline.config
sed -i "s|num_classes: [[:digit:]]\+|num_classes: ${NUM_CLASS}|g" ${MODEL_OUT_PATH}/pipeline.config

sed -i "s|.record\"|.record-?????-of-00010\"|g" ${MODEL_OUT_PATH}/pipeline.config 
sed -i "s|step: 0|step: 1|g" ${MODEL_OUT_PATH}/pipeline.config # Bug fix 

# Check the final config manually
#cat ${MODEL_OUT_PATH}/pipeline.config

In [ ]:
%env MODEL_NAME=ssd_mobilenet_v1_coco

#export MODEL_NAME='ssd_mobilenet_v1_coco'
#export MODEL_NAME='faster_rcnn_resnet101_coco'
#export MODEL_NAME='ssd_inception_v2_coco'

!python ${TF_OBJECT_DETECTION_API}/model_main.py \
    --model_dir=/data/pets/trained_model/${MODEL_NAME} \
    --pipeline_config_path=/data/pets/trained_model/pipeline.config \
    --alsologtostderr \
    --num_train_step=50000
    